In [ ]:
# Testing loading data from mongo db into clean dataframes

In [4]:
# Imports
import pandas as pd
import numpy as np
import json

In [5]:
# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# Mongo database and collection specification:
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.btcusd_db
input_data = db.btcusd_collection
data = pd.DataFrame(list(input_data.find()))

In [10]:
data.head()

,_id,asks,bids,changes,channels,maker_order_id,message,price,product_id,reason,sequence,side,size,taker_order_id,time,trade_id,type
0,5ad3dc2ee3ae711d3ca939a4,"[[8337.67, 35.73324449], [8337.7, 1], [8337.71...","[[8337.66, 1.92259307], [8337, 0.001], [8336, ...",NaN,NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,snapshot
1,5ad3dc2ee3ae711d3ca939a5,NaN,NaN,NaN,NaN,97e82bf6-c071-48d7-9810-0cf52e9a3041,NaN,8337.66000000,BTC-USD,NaN,5.686020e+09,buy,0.23590790,f87435c9-9d7d-485b-b52a-ca76b0bec88d,2018-04-15T23:11:40.934000Z,41700565.0,last_match
2,5ad3dc2ee3ae711d3ca939a6,NaN,NaN,NaN,"[{'name': 'level2', 'product_ids': ['BTC-USD']...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,subscriptions
3,5ad3dc2ee3ae711d3ca939a7,NaN,NaN,"[[buy, 8300.00000000, 33.48472626]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-15T23:11:43.047Z,NaN,l2update
4,5ad3dc2ee3ae711d3ca939a8,NaN,NaN,"[[sell, 8353.46000000, 0.046]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-15T23:11:43.060Z,NaN,l2update
